#Import Data from Kaggle#

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
 
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d brsdincer/wildfire-detection-image-data

 36% 9.00M/24.9M [00:00<00:00, 28.6MB/s]
100% 24.9M/24.9M [00:00<00:00, 56.2MB/s]


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  wildfire-detection-image-data.zip
  inflating: forest_fire/Testing/fire/abc169.jpg  
  inflating: forest_fire/Testing/fire/abc170.jpg  
  inflating: forest_fire/Testing/fire/abc171.jpg  
  inflating: forest_fire/Testing/fire/abc172.jpg  
  inflating: forest_fire/Testing/fire/abc173.jpg  
  inflating: forest_fire/Testing/fire/abc174.jpg  
  inflating: forest_fire/Testing/fire/abc175.jpg  
  inflating: forest_fire/Testing/fire/abc176.jpg  
  inflating: forest_fire/Testing/fire/abc177.jpg  
  inflating: forest_fire/Testing/fire/abc178.jpg  
  inflating: forest_fire/Testing/fire/abc179.jpg  
  inflating: forest_fire/Testing/fire/abc180.jpg  
  inflating: forest_fire/Testing/fire/abc181.jpg  
  inflating: forest_fire/Testing/fire/abc182.jpg  
  inflating: forest_fire/Testing/fire/abc183.jpg  
  inflating: forest_fire/Testing/fire/abc184.jpg  
  inflating: forest_fire/Testing/fire/abc185.jpg  
  inflating: forest_fire/Testing/fire/abc186.jpg  
  inflating: forest_fire/Testing/fire/

#Data Generator#

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        shear_range=0.3,
        zoom_range=0.3
        )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/forest_fire/Training and Validation',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary',
        color_mode = 'grayscale'
        )

validation_generator = test_datagen.flow_from_directory(
        '/content/forest_fire/Testing',
        target_size=(256, 256),
        batch_size=32,
        class_mode='binary',
        color_mode = 'grayscale'
        )

Found 1832 images belonging to 2 classes.
Found 68 images belonging to 2 classes.


#Model#

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout

In [ ]:
model = Sequential() 

model.add(Conv2D(128, (3, 3), padding="same", activation="relu", input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=256, activation="relu"))

model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 256, 256, 128)     1280      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 64)      73792     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 64)       

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1 , patience=75)

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit_generator(train_generator, validation_data = validation_generator, epochs=100, callbacks=es)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
58/58 [==============================] - 41s 179ms/step - loss: 0.6862 - accuracy: 0.5636 - val_loss: 0.5794 - val_accuracy: 0.6912
Epoch 2/100
58/58 [==============================] - 10s 174ms/step - loss: 0.5150 - accuracy: 0.7617 - val_loss: 0.3882 - val_accuracy: 0.8235
Epoch 3/100
58/58 [==============================] - 10s 174ms/step - loss: 0.3550 - accuracy: 0.8395 - val_loss: 0.3007 - val_accuracy: 0.8824
Epoch 4/100
58/58 [==============================] - 10s 174ms/step - loss: 0.3685 - accuracy: 0.8345 - val_loss: 0.3018 - val_accuracy: 0.8971
Epoch 5/100
58/58 [==============================] - 10s 174ms/step - loss: 0.3086 - accuracy: 0.8652 - val_loss: 0.2739 - val_accuracy: 0.8824
Epoch 6/100
58/58 [==============================] - 10s 174ms/step - loss: 0.2474 - accuracy: 0.9006 - val_loss: 0.2584 - val_accuracy: 0.8971
Epoch 7/100
58/58 [==============================] - 10s 175ms/step - loss: 0.2919 - accuracy: 0.8644 - val_loss: 0.2456 - val_accuracy:

In [ ]:
#Get class indices for later use in flask web app.
train_generator.class_indices

{'fire': 0, 'nofire': 1}

In [ ]:
#Download the model as an h5 file to use in flask web app.
model.save("wildfire_detection.h5")